# Initialization


## Installation and Importation

In [1]:
!pip install pandasql
!pip install pandas
!pip install plotly
!pip install optimparallel
!pip install torch
!pip install tensorflow
!pip install tensorflow_probability
!pip install pytorch-minimize
!pip install beautifulsoup4
!pip install pyopenssl
!pip install aia
!pip install numdifftools
!pip install sklearn
!pip install tensorflow


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip



In [2]:
import pandas as pd
import numpy as np
import functools
import pickle

from BB_solver import bb_ll
from BB_solver import pre_obj_fn

from bs4 import BeautifulSoup
from aia import AIASession
from urllib.request import urlopen
from scipy.optimize import Bounds
from optimparallel import minimize_parallel

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as XGBoost
import tensorflow as tf

from random import random


# Data Wrangling

In [3]:
year = 2019
box_scores_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/Data/ncaam{0}.csv'.format(year))
last_day_before_tournament = 135
recency_denom = 15

## Constructing Main Dataframe

Make box scores dataframe with added shooting percentage columns

In [4]:
# make shooting percentage columns
box_scores_df['FG%'] = box_scores_df['FGM'] / box_scores_df['FGA']
box_scores_df['3FG%'] = box_scores_df['3FGM'] / box_scores_df['3FGA']
box_scores_df['FT%'] = box_scores_df['FTM'] / box_scores_df['FTA']

box_scores_df['EFG_N'] = round(box_scores_df['FGM'] + 0.5 * box_scores_df['3FGM'], 0).astype('int')
box_scores_df['EFG_D'] = round(box_scores_df['FGA'], 0).astype('int')

box_scores_df['EFG2_N'] = round(box_scores_df['FGM'] - box_scores_df['3FGM'], 0).astype('int')
box_scores_df['EFG2_D'] = round(box_scores_df['FGA'] - box_scores_df['3FGA'], 0).astype('int')

box_scores_df['EFG3_N'] = round(box_scores_df['3FGM'], 0).astype('int')
box_scores_df['EFG3_D'] = round(box_scores_df['3FGA'], 0).astype('int')

box_scores_df['EFT_N'] = round(box_scores_df['FTM'], 0).astype('int')
box_scores_df['EFT_D'] = round(box_scores_df['FGA'], 0).astype('int')
box_scores_df['EFT_N'] = box_scores_df.apply(lambda row: min(row['EFT_N'], row['EFT_D']), axis=1).astype('int')

box_scores_df['EAST_N'] = round(box_scores_df['AST'], 0).astype('int')
box_scores_df['EAST_D'] = round(box_scores_df['FGM'], 0).astype('int')

In [5]:
box_scores_df

,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,EFG_N,EFG_D,EFG2_N,EFG2_D,EFG3_N,EFG3_D,EFT_N,EFT_D,EAST_N,EAST_D
0,1,1380,1104,Alabama,1,0,0,0.0,82,27.0,...,32,55,18,31,9,24,19,55,17,27
1,1,1104,1380,Southern Univ,1,0,0,0.0,62,21.0,...,22,55,18,38,3,17,17,55,8,21
2,2,1378,1105,Alabama A&M,1,0,0,0.0,63,22.0,...,26,52,13,27,9,25,10,52,18,22
3,2,1105,1378,South Florida,1,0,0,0.0,80,30.0,...,34,53,21,32,9,21,11,53,20,30
4,3,1235,1106,Alabama St,1,0,0,0.0,53,20.0,...,24,55,11,29,9,26,4,55,11,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,5528,1120,1438,Virginia,152,0,1,0.0,63,25.0,...,28,51,18,32,7,19,6,51,15,25
11056,5529,1277,1403,Texas Tech,152,0,1,0.0,61,22.0,...,26,51,13,28,9,23,8,51,8,22
11057,5529,1403,1277,Michigan St,152,0,1,0.0,51,15.0,...,18,47,8,23,7,24,14,47,6,15
11058,5530,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,32,63,17,33,10,30,13,63,9,27


Include opponent data

In [6]:
# make a copy of the box scores data with flipped binary 'Home' column values
#    (0 -> 1 and 1 -> 0)
box_scores_home_away_flipped_df = box_scores_df.copy()

box_scores_home_away_flipped_df['Home'] = \
  box_scores_home_away_flipped_df['Home'] \
  .replace(1, -1).replace(0, 1).replace(-1, 0)

# since this houses the soon to be opponent data, rename 'Team' as 'Opponent'
box_scores_home_away_flipped_df = \
  box_scores_home_away_flipped_df.rename(columns={"Team":"Opponent"})

In [7]:
# merge box score data and box scores flipped dataframes
#    (team and opponent data sets) on the home team
home_games_df = \
  box_scores_df.merge(box_scores_home_away_flipped_df, how='inner',
                      left_on=['GameId', 'Home', 'DayNum', 'Team'],
                      right_on=['GameId', 'Home', 'DayNum', 'HomeTeam'])

# for the home team data, rename the opponent columns with '_Opp'
#    suffix convention
for col in home_games_df.columns:
  if '_x' in col:
    home_games_df = \
      home_games_df.rename(columns={col: col.replace('_x', '')})

  elif '_y' in col:
    home_games_df = \
      home_games_df.rename(columns={col: col.replace('_y', '_Opp')})

# remove unneccesary columns from home team data
home_games_df = home_games_df.drop(columns=['HomeTeam', 'AwayTeam',
                                            'HomeTeam_Opp', 'AwayTeam_Opp',
                                            'NeutralSite_Opp'])

# merge box score data and box scores flipped dataframes
#    (team and opponent data sets) on the away team
away_games_df = \
  box_scores_df.merge(box_scores_home_away_flipped_df,how='inner',
                      left_on=['GameId', 'Home', 'DayNum', 'Team'],
                      right_on=['GameId', 'Home', 'DayNum', 'AwayTeam'])

# for the away team data, rename the opponent columns with '_Opp'
#    suffix convention
for col in away_games_df.columns:
  if '_x' in col:
    away_games_df = \
      away_games_df.rename(columns={col: col.replace('_x', '')})

  elif '_y' in col:
    away_games_df = \
      away_games_df.rename(columns={col: col.replace('_y', '_Opp')})

# remove unneccesary columns from away team data
away_games_df = \
  away_games_df.drop(columns=['HomeTeam', 'AwayTeam', 'HomeTeam_Opp',
                              'AwayTeam_Opp', 'NeutralSite_Opp'])

In [8]:
# concatenate home and away team dataframes into a new all games dataframe
all_games_df = pd.concat([home_games_df, away_games_df])

# make a binary win column taking 1 (win) or 0 (loss)
all_games_df['Win'] = \
  np.sign(all_games_df['Score'] - all_games_df['Score_Opp']).replace(-1, 0)

# sort all games dataframe on 'GameId' (first, ascending) and
#    'Home' (second, descending)
all_games_df = \
  all_games_df.sort_values(["GameId", "Home"], ascending = [True, False])

Make offensive and defensive rebounding factor columns

In [9]:
# make last of the factors columns (effective turnover rate,
#    effective rebound rates, which will be be split into 2 fields
#    (offensive and defensive rebounds)
all_games_df['ETOV_N'] = round(all_games_df['TOV_Opp'], 0).astype('int')
all_games_df['ETOV_N_Opp'] = round(all_games_df['TOV'], 0).astype('int')

all_games_df['ETOV_D'] = round(all_games_df['FGA_Opp'] + 0.44 * all_games_df['FTA_Opp'] + all_games_df['TOV_Opp']).astype('int')
all_games_df['ETOV_D_Opp'] = round(all_games_df['FGA'] + 0.44 * all_games_df['FTA'] + all_games_df['TOV']).astype('int')

# make effective offensive rebound columns (for team and opponent)
all_games_df['EORB_N'] = round(all_games_df['ORB'], 0).astype('int')
all_games_df['EORB_D'] = round(all_games_df['ORB'] + all_games_df['DRB_Opp'], 0).astype('int')

all_games_df['EORB_N_Opp'] = round(all_games_df['ORB_Opp'], 0).astype('int')
all_games_df['EORB_D_Opp'] = round(all_games_df['ORB_Opp'] + all_games_df['DRB'], 0).astype('int')

# make effective defensive rebound columns (for team and opponent)
all_games_df['EDRB_N'] = round(all_games_df['DRB'], 0).astype('int')
all_games_df['EDRB_D'] = round(all_games_df['DRB'] + all_games_df['ORB_Opp'], 0).astype('int')

all_games_df['EDRB_N_Opp'] = round(all_games_df['DRB_Opp'], 0).astype('int')
all_games_df['EDRB_D_Opp'] = round(all_games_df['DRB_Opp'] + all_games_df['ORB'], 0).astype('int')

# make effctive blocks columns
all_games_df['EBLK_N'] = round(all_games_df['BLK'], 0).astype('int')
all_games_df['EBLK_D'] = round(all_games_df['FGA_Opp'], 0).astype('int')

all_games_df['EBLK_N_Opp'] = round(all_games_df['BLK_Opp'], 0).astype('int')
all_games_df['EBLK_D_Opp'] = round(all_games_df['FGA'], 0).astype('int')

# regarding the home column, make away games -1 and neutral site games 0
all_games_df['Home'] = all_games_df['Home'].replace(0, -1)

all_games_df['Home'] = \
  np.where(all_games_df['NeutralSite'] == 1, 0, all_games_df['Home'])

all_games_df['Tourney'] = all_games_df.apply(lambda x: x['ConfTourney'] + x['MMTourney'], axis=1)

In [10]:
all_games_df['DayNumAdj'] = all_games_df['DayNum'].apply(lambda x: np.abs(x-last_day_before_tournament-1)**(-1/recency_denom)
                            if x-last_day_before_tournament <= 0 else None)

In [11]:
all_games_df = \
  all_games_df.drop(columns=['Conf_Opp', 'OppConf_Opp', 'OppTeamID_Opp',
                             'TeamID_Opp', 'ConfTourney', 'MMTourney',
                             'FGM', 'FGA', '3FGM', '3FGA', 'FTM',
                             'FTA', 'ORB', 'DRB', 'AST', 'TOV', 'STL', 'BLK',
                             'Fouls', 'FG%', '3FG%', 'FT%', 'ConfTourney_Opp',
                             'MMTourney_Opp', 'NumOT_Opp', 'FGM_Opp', 'FGA_Opp',
                             '3FGM_Opp', '3FGA_Opp', 'FTM_Opp', 'FTA_Opp',
                             'ORB_Opp', 'DRB_Opp', 'AST_Opp', 'TOV_Opp',
                             'STL_Opp', 'BLK_Opp', 'Fouls_Opp', 'FG%_Opp',
                             '3FG%_Opp', 'FT%_Opp', 'OppTeamID', 'TeamID',
                             'NeutralSite'])

## Encoding Team Names

In [12]:
codes, uniques = pd.factorize(all_games_df['Team'], sort=True)
teams_dictionary = dict(zip(uniques, [_ for _ in range(len(uniques))]))
inv_map = {v: k for k, v in teams_dictionary.items()}

In [13]:
def bin_score_diff(diff):
    if abs(diff) <= 3:
        if np.sign(diff) == 1:
            return '0 < diff <= 3'
        else:
            return '-3 <= diff < 0'

    if abs(diff) <= 6:
        if np.sign(diff) == 1:
            return '3 < diff <= 6'
        else:
            return '-6 <= diff < -3'

    if abs(diff) <= 10:
        if np.sign(diff) == 1:
            return '6 < diff <= 10'
        else:
            return '-10 <= diff < -6'

    if abs(diff) <= 15:
        if np.sign(diff) == 1:
            return '10 < diff <= 15'
        else:
            return '-15 <= diff < -10'

    if abs(diff) <= 20:
        if np.sign(diff) == 1:
            return '15 < diff <= 20'
        else:
            return '-20 <= diff < -15'

    if abs(diff) > 20:
        if np.sign(diff) == 1:
            return 'diff > 20'
        else:
            return 'diff < -20'

In [14]:
games_df = all_games_df[['Team', 'Opponent', 'Score', 'Score_Opp', 'Home', 'Tourney', 'DayNum', 'DayNumAdj']]
games_df = games_df[games_df['DayNum'] <= last_day_before_tournament]

games_df = games_df.reset_index()
games_df['Score'] = games_df['Score'] - games_df['Score_Opp']
games_df = games_df.drop(columns=['Score_Opp'])
games_df = games_df.rename(columns={'Score':'ScoreDiff'})
games_df['Home'] = games_df.apply(lambda row: row['Home'] if row['ScoreDiff'] > 0 else (-row['Home'] if row['ScoreDiff'] < 0 else 0), axis=1)
games_df['Winner'] = games_df.apply(lambda row: row['Team'] if row['ScoreDiff'] > 0 else row['Opponent'], axis=1)
games_df['Loser'] = games_df.apply(lambda row: row['Team'] if row['ScoreDiff'] < 0 else row['Opponent'], axis=1)
games_df['ScoreDiff'] = np.abs(games_df['ScoreDiff'])
games_df = games_df[['index', 'Winner', 'Loser', 'ScoreDiff', 'Home', 'Tourney', 'DayNum', 'DayNumAdj']]

games_df = games_df[games_df.index % 2 == 0]
games_df = games_df.drop(columns='index')

teams_set = set(all_games_df['Team'].tolist())
teams_list = list(teams_set)
teams_list.sort()

games_df = games_df.replace({"Winner": teams_dictionary})
games_df = games_df.replace({"Loser": teams_dictionary})

games_df['BinnedOutcome'] = games_df['ScoreDiff'].apply(bin_score_diff)

games_array = games_df.to_numpy()
games_df

,Winner,Loser,ScoreDiff,Home,Tourney,DayNum,DayNumAdj,BinnedOutcome
0,3,271,20,1,0,1,0.721071,15 < diff <= 20
2,269,4,17,1,0,1,0.721071,15 < diff <= 20
4,127,5,26,1,0,1,0.721071,diff > 20
6,301,6,17,1,0,1,0.721071,15 < diff <= 20
8,10,36,8,1,0,1,0.721071,6 < diff <= 10
...,...,...,...,...,...,...,...,...
10924,278,273,2,0,1,132,0.911722,0 < diff <= 3
10926,21,289,11,0,1,134,0.954842,10 < diff <= 15
10928,84,231,6,0,1,134,0.954842,3 < diff <= 6
10930,10,276,9,0,1,135,1.000000,6 < diff <= 10


In [15]:
set(games_df['BinnedOutcome'])

{'0 < diff <= 3',
 '10 < diff <= 15',
 '15 < diff <= 20',
 '3 < diff <= 6',
 '6 < diff <= 10',
 'diff > 20'}

# Get Initial Rankings

These will be used for weighting games when solving for alpha and beta values across various metrics

In [16]:
def pre_solver(teams_list, winners, losers, home, binned_outcome):

    new_pre_obj_fn = functools.partial(pre_obj_fn, args=[winners, losers, home, binned_outcome])
    x0 = [1.0 for _ in range(len(teams_list))]

    x0.append(1.0)
    x0 = tf.convert_to_tensor(x0, tf.float64)

    result = minimize_parallel(fun=new_pre_obj_fn, x0=x0, parallel={'forward':False, 'max_workers': 10})

    return result

In [17]:
result = pre_solver(teams_list, np.array(games_df['Winner']), np.array(games_df['Loser']), np.array(games_df['Home']),
                    np.array(games_df['BinnedOutcome']))
print("Loss:", result.fun)
result.x

Loss: 1.609096828779901


array([ 4.17773081e-01, -1.28763797e+00,  5.60504613e+00,  4.91250490e+00,
       -6.06722957e+00, -5.92004142e+00, -9.73102383e+00,  1.51078854e+00,
        1.21659846e+00,  4.78104383e+00,  5.02714850e+00,  2.07380014e+00,
       -5.10789343e+00,  6.91003318e+00, -2.92162491e+00, -1.65062795e+00,
        9.50876507e+00,  1.69607274e+00,  3.20554154e+00,  3.70929505e+00,
        6.79415063e+00,  4.50366262e+00, -2.84471436e+00, -5.05655189e+00,
        6.59397235e+00,  3.89758750e+00, -1.27559099e+00,  2.68739904e+00,
       -4.19475795e-01,  1.55521468e+00, -6.20176211e+00,  8.58743175e-01,
        7.24660894e+00,  5.80929202e+00,  2.16004371e+00, -7.06878818e-01,
        1.49555761e-01, -6.58271727e-01, -2.07384762e+00,  8.89434464e-02,
       -4.60997455e+00, -1.10194614e+00, -2.90599182e-01, -4.22706780e+00,
       -3.52175176e+00, -5.31750755e+00,  2.31205851e+00, -3.27089725e+00,
       -2.38616004e+00, -8.48433888e+00,  5.47006134e+00, -3.02917017e+00,
        8.61165265e+00, -

In [18]:
ratings = pd.DataFrame(data=result.x[:-1], columns=['Rating'])
ratings['Team'] = [_ for _ in range(len(result.x[:-1]))]
ratings = ratings[['Team', 'Rating']]
ratings_std = ratings['Rating'].std()
ratings['Rating'] -= ratings['Rating'].mean()
ratings['Rating'] /= ratings_std
ratings = ratings.replace({'Team': inv_map})
ratings = ratings.sort_values(by='Rating', ascending=False)
teams_dict = ratings.set_index('Team').to_dict()['Rating']
ratings

,Team,Rating
170,Michigan St,2.697715
103,Gonzaga,2.675392
329,Virginia,2.593714
74,Duke,2.589946
169,Michigan,2.431137
...,...,...
49,Chicago St,-2.086918
155,MS Valley St,-2.140746
6,Alcorn St,-2.361245
154,MD E Shore,-2.590425


In [19]:
all_games_df['Rating'] = all_games_df.apply(lambda row : teams_dict[row['Team']], axis=1)
all_games_df['OppRating'] = all_games_df.apply(lambda row : teams_dict[row['Opponent']], axis=1)

all_games_df['AvgRating'] = all_games_df.apply(lambda row: (row['Rating'] + row['OppRating']) / 2, axis=1)
all_games_df['RatingAdj'] = all_games_df['AvgRating'].apply(lambda x: 2/(1+np.exp(-x)))

In [20]:
all_games_df = all_games_df.replace({"Team": teams_dictionary})
all_games_df = all_games_df.replace({"Opponent": teams_dictionary})
all_games_df = all_games_df.replace({"Team": teams_dictionary})
all_games_df = all_games_df.replace({"Opponent": teams_dictionary})
all_games_df

,GameId,Team,DayNum,NumOT,Score,Home,Conf,OppConf,Seed,EFG_N,...,EBLK_N,EBLK_D,EBLK_N_Opp,EBLK_D_Opp,Tourney,DayNumAdj,Rating,OppRating,AvgRating,RatingAdj
0,1,3,1,0.0,82,1,sec,swac,NaN,32,...,6,55,2,55,0,0.721071,0.860997,-1.074899,-0.106951,0.946575
0,1,271,1,0.0,62,-1,swac,sec,NaN,22,...,2,55,6,55,0,0.721071,-1.074899,0.860997,-0.106951,0.946575
1,2,269,1,0.0,80,1,aac,swac,NaN,34,...,4,52,0,53,0,0.721071,0.407034,-1.555044,-0.574005,0.720626
1,2,4,1,0.0,63,-1,swac,aac,NaN,26,...,0,53,4,52,0,0.721071,-1.555044,0.407034,-0.574005,0.720626
2,3,127,1,0.0,79,1,big_twelve,swac,6.0,37,...,5,55,3,67,0,0.721071,1.959384,-1.522656,0.218364,1.108750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527,5528,16,152,0.0,62,0,sec,acc,5.0,26,...,3,51,9,55,1,NaN,1.872383,2.593714,2.233049,1.806357
5528,5529,296,152,0.0,61,0,big_twelve,big_ten,3.0,26,...,4,47,2,51,1,NaN,1.981182,2.697715,2.339449,1.824184
5528,5529,170,152,0.0,51,0,big_ten,big_twelve,2.0,18,...,2,51,4,47,1,NaN,2.697715,1.981182,2.339449,1.824184
5529,5530,329,154,1.0,85,0,acc,big_twelve,1.0,32,...,3,63,3,59,1,NaN,2.593714,1.981182,2.287448,1.815664


# Beta-Binomial Model

## Alpha-Beta Solver

In [21]:
def ab_solver(winners, losers, loc, metric_n, metric_d, recency_weights, ratings_weights, x0=None):

    teams_n = len(set(winners))

    if x0 is None:
        alphas = [100.0 for _ in range(teams_n)]
        betas = [200.0 for _ in range(teams_n)]
        home_alpha = [10.0]
        neutral_alpha = [5.0]
        home_beta = [10.0]
        neutral_beta = [5.0]
        x0 = alphas + betas + home_alpha + neutral_alpha + home_beta + neutral_beta

    obj_fn = functools.partial(bb_ll, args=[winners, losers, loc, metric_n, metric_d, recency_weights, ratings_weights, teams_n])

    lower_ab = [0.001 for _ in range(2*teams_n)]
    lower_alpha_ha = [0, 0, 0, 0]
    lower = lower_ab + lower_alpha_ha

    upper_ab = [np.inf for _ in range(2*teams_n)]
    upper_alpha_ha = [np.inf, np.inf, np.inf, np.inf]
    upper = upper_ab + upper_alpha_ha

    bounds = Bounds(lower, upper)

    result = minimize_parallel(fun=obj_fn, x0=x0, parallel={'max_workers': 10, 'forward':False}, bounds=bounds)

    return result

In [22]:
def ab_populator(games_df, metrics, x_dic, loc_dic, last_pre_tour_game):

    train_df = games_df[games_df['DayNum'] <= last_pre_tour_game]

    for iteration in range(3):
        print('----------------------------------------')
        print("Iteration:", iteration+1)
        print('----------------------------------------')
        for metric in metrics:

            alphas = x_vals_dict.get(metric+'_alphas', None)
            betas = x_vals_dict.get(metric+'_betas', None)
            home_alpha_weight = home_adv_dict.get(metric+'_home_alpha', None)
            neutral_alpha_weight = home_adv_dict.get(metric+'_neutral_alpha', None)
            home_beta_weight = home_adv_dict.get(metric+'_home_beta', None)
            neutral_beta_weight = home_adv_dict.get(metric+'_neutral_beta', None)

            if (alphas is not None) and (betas is not None) and (home_alpha_weight is not None) and\
                    (neutral_alpha_weight is not None) and (home_beta_weight is not None) and\
                    (neutral_beta_weight is not None):
                x0 = list(alphas) + list(betas) + [home_alpha_weight] + [neutral_alpha_weight] +\
                     [home_beta_weight] + [neutral_beta_weight]
            else:
                x0 = None

            n_data = train_df[metric+'_N'].to_list()
            d_data = train_df[metric+'_D'].to_list()

            result = ab_solver(train_df['Team'].to_list(), train_df['Opponent'].to_list(), train_df['Home'].to_list(),
                               np.array(n_data), np.array(d_data), np.array(train_df['DayNumAdj'].to_list()),
                               np.array(train_df['RatingAdj'].to_list()), x0)

            n_teams = len(set(train_df['Team'].to_list()))
            print(metric, ": ", "converged" if result.success else "failed", " | Loss: ", round(result.fun, 2), sep="")

            alpha = result.x[0:n_teams]
            beta = result.x[n_teams:2*n_teams]
            home_alpha_weight = result.x[-4]
            neutral_alpha_weight = result.x[-3]
            home_beta_weight = result.x[-2]
            neutral_beta_weight = result.x[-1]

            x_dic[metric+'_alphas'] = alpha
            x_dic[metric+'_betas'] = beta
            loc_dic[metric+'_home_alpha'] = home_alpha_weight
            loc_dic[metric+'_neutral_alpha'] = neutral_alpha_weight
            loc_dic[metric+'_home_beta'] = home_beta_weight
            loc_dic[metric+'_neutral_beta'] = neutral_beta_weight

    for metric in metrics:
        games_df[metric+'_alphas_team1'] = games_df.apply(lambda row: x_dic[metric+'_alphas'][row['Team']] +
                                                                      (loc_dic[metric+'_home_alpha'] if row['Home'] == 1 else
                                                                       (loc_dic[metric+'_neutral_alpha'] if row['Home'] == 0 else 0)), axis=1)

        games_df[metric+'_betas_team1'] = games_df.apply(lambda row: x_dic[metric+'_betas'][row['Team']] +
                                                                     (loc_dic[metric+'_home_beta'] if row['Home'] == 1 else
                                                                      (loc_dic[metric+'_neutral_beta'] if row['Home'] == 0 else 0)), axis=1)

        games_df[metric+'_alphas_team2'] = games_df.apply(lambda row: x_dic[metric+'_alphas'][row['Opponent']] +
                                                                      (loc_dic[metric+'_home_alpha'] if row['Home'] == -1 else
                                                                       (loc_dic[metric+'_neutral_alpha'] if row['Home'] == 0 else 0)), axis=1)

        games_df[metric+'_betas_team2'] = games_df.apply(lambda row: x_dic[metric+'_betas'][row['Opponent']] +
                                                                     (loc_dic[metric+'_home_beta'] if row['Home'] == -1 else
                                                                      (loc_dic[metric+'_neutral_beta'] if row['Home'] == 0 else 0)), axis=1)

        games_df[metric+'_avgs_team1'] = games_df.apply(lambda row: row[metric+'_alphas_team1'] /
                                                                    (row[metric+'_alphas_team1'] + row[metric+'_betas_team2']), axis=1)

        games_df[metric+'_avgs_team2'] = games_df.apply(lambda row: row[metric+'_alphas_team2'] /
                                                                    (row[metric+'_alphas_team2'] + row[metric+'_betas_team1']), axis=1)

    games_df['EDRB_avgs_team1'] = games_df.apply(lambda row: row['EORB_betas_team1'] /
                                                                (row['EORB_betas_team1'] + row['EORB_alphas_team2']), axis=1)

    games_df['EDRB_avgs_team2'] = games_df.apply(lambda row: row['EORB_betas_team2'] /
                                                                (row['EORB_betas_team2'] + row['EORB_alphas_team1']), axis=1)

In [23]:
metrics = ['EFG', 'EFG2', 'EFG3', 'EFT', 'EORB', 'EAST', 'ETOV', 'EBLK']

try:
    with open('Data/{0}_x_vals_lr_dict.pkl'.format(year), 'rb') as f:
        x_vals_dict = pickle.load(f)

    with open('Data/{0}_home_adv_lr_dict.pkl'.format(year), 'rb') as f:
        home_adv_dict = pickle.load(f)

    for metric in metrics:
        all_games_df[metric+'_alphas_team1'] = all_games_df.apply(lambda row: x_vals_dict[metric+'_alphas'][row['Team']] +
                                                                              (home_adv_dict[metric+'_home_alpha'] if row['Home'] == 1 else
                                                                               (home_adv_dict[metric+'_neutral_alpha'] if row['Home'] == 0 else 0)),
                                                                  axis=1)

        all_games_df[metric+'_betas_team1'] = all_games_df.apply(lambda row: x_vals_dict[metric+'_betas'][row['Team']] +
                                                                             (home_adv_dict[metric+'_home_beta'] if row['Home'] == 1 else
                                                                              (home_adv_dict[metric+'_neutral_beta'] if row['Home'] == 0 else 0)), axis=1)

        all_games_df[metric+'_alphas_team2'] = all_games_df.apply(lambda row: x_vals_dict[metric+'_alphas'][row['Opponent']] +
                                                                              (home_adv_dict[metric+'_home_alpha'] if row['Home'] == -1 else
                                                                               (home_adv_dict[metric+'_neutral_alpha'] if row['Home'] == 0 else 0)),
                                                                  axis=1)

        all_games_df[metric+'_betas_team2'] = all_games_df.apply(lambda row: x_vals_dict[metric+'_betas'][row['Opponent']] +
                                                                             (home_adv_dict[metric+'_home_beta'] if row['Home'] == -1 else
                                                                              (home_adv_dict[metric+'_neutral_beta'] if row['Home'] == 0 else 0)), axis=1)

        all_games_df[metric+'_avgs_team1'] = all_games_df.apply(lambda row: row[metric+'_alphas_team1'] /
                                                                            (row[metric+'_alphas_team1'] + row[metric+'_betas_team2']), axis=1)

        all_games_df[metric+'_avgs_team2'] = all_games_df.apply(lambda row: row[metric+'_alphas_team2'] /
                                                                            (row[metric+'_alphas_team2'] + row[metric+'_betas_team1']), axis=1)

    all_games_df['EDRB_avgs_team1'] = all_games_df.apply(lambda row: row['EORB_betas_team1'] /
                                                             (row['EORB_betas_team1'] + row['EORB_alphas_team2']), axis=1)

    all_games_df['EDRB_avgs_team2'] = all_games_df.apply(lambda row: row['EORB_betas_team2'] /
                                                             (row['EORB_betas_team2'] + row['EORB_alphas_team1']), axis=1)

except FileNotFoundError:
    x_vals_dict = dict()
    home_adv_dict = dict()
    ab_populator(all_games_df, metrics, x_vals_dict, home_adv_dict, last_day_before_tournament)

    with open('Data/{0}_x_vals_lr_dict.pkl'.format(year), 'wb') as f:
        pickle.dump(x_vals_dict, f)

    with open('Data/{0}_home_adv_lr_dict.pkl'.format(year), 'wb') as f:
        pickle.dump(home_adv_dict, f)

----------------------------------------
Iteration: 1
----------------------------------------
EFG: converged | Loss: 24280.46
EFG2: converged | Loss: 20736.48
EFG3: converged | Loss: 18376.53
EFT: converged | Loss: 26338.6
EORB: converged | Loss: 20141.52
EAST: converged | Loss: 20822.55
ETOV: converged | Loss: 21776.41
EBLK: converged | Loss: 16308.2
----------------------------------------
Iteration: 2
----------------------------------------
EFG: converged | Loss: 24280.46
EFG2: converged | Loss: 20723.87
EFG3: converged | Loss: 18375.81
EFT: converged | Loss: 26338.6
EORB: converged | Loss: 20141.49
EAST: converged | Loss: 20822.55
ETOV: converged | Loss: 21776.4
EBLK: converged | Loss: 16308.03
----------------------------------------
Iteration: 3
----------------------------------------
EFG: converged | Loss: 24280.46
EFG2: converged | Loss: 20723.76
EFG3: converged | Loss: 18375.81
EFT: converged | Loss: 26338.6
EORB: converged | Loss: 20141.48
EAST: converged | Loss: 20822.55


Check that this date corresponds to the four play-in games

In [24]:
all_games_df[(all_games_df['DayNum'] >= last_day_before_tournament-1) & (all_games_df['DayNum'] <= last_day_before_tournament)][['GameId', 'DayNum', 'Team', 'Opponent', 'Win']]

,GameId,DayNum,Team,Opponent,Win
5463,5464,134,289,21,0
5463,5464,134,21,289,1
5464,5465,134,231,84,0
5464,5465,134,84,231,1
5465,5466,135,276,10,0
5465,5466,135,10,276,1
5466,5467,135,189,185,0
5466,5467,135,185,189,1


# Save the Results

In [25]:
compression_opts = dict(method='zip', archive_name='NCAA_df_{0}.csv'.format(str(year)[-2:]))
all_games_df.to_csv('NCAA_df_{0}.zip'.format(str(year)[-2:]), index=False, compression=compression_opts)

# Reinstate Team Names

In [59]:
inv_map = {v: k for k, v in teams_dictionary.items()}
all_games_df = all_games_df.replace({"Team": inv_map})
all_games_df = all_games_df.replace({"Opponent": inv_map})
all_games_df

,GameId,Team,DayNum,NumOT,Score,Home,Conf,OppConf,Seed,EFG_N,...,ETOV_avgs_team1,ETOV_avgs_team2,EBLK_alphas_team1,EBLK_betas_team1,EBLK_alphas_team2,EBLK_betas_team2,EBLK_avgs_team1,EBLK_avgs_team2,EDRB_avgs_team1,EDRB_avgs_team2
0,1,Alabama,1,0.0,82,1,sec,swac,NaN,32,...,0.198659,0.176691,70.857967,769.755656,31.713175,520.838352,0.119754,0.039569,0.760160,0.628058
0,1,Southern Univ,1,0.0,62,-1,swac,sec,NaN,22,...,0.176691,0.198659,31.713175,520.838352,70.857967,769.755656,0.039569,0.119754,0.628058,0.760160
1,2,South Florida,1,0.0,80,1,aac,swac,NaN,34,...,0.243516,0.204495,49.328668,635.022225,18.774461,582.000176,0.078135,0.028716,0.767869,0.580400
1,2,Alabama A&M,1,0.0,63,-1,swac,aac,NaN,26,...,0.204495,0.243516,18.774461,582.000176,49.328668,635.022225,0.028716,0.078135,0.580400,0.767869
2,3,Iowa St,1,0.0,79,1,big_twelve,swac,6.0,37,...,0.188177,0.147219,67.164063,1130.381158,37.225651,509.346441,0.116501,0.031882,0.709813,0.649918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527,5528,Auburn,152,0.0,62,0,sec,acc,5.0,26,...,0.169300,0.145594,80.198407,960.833364,63.842209,980.463322,0.075612,0.062305,0.681012,0.718342
5528,5529,Texas Tech,152,0.0,61,0,big_twelve,big_ten,3.0,26,...,0.201261,0.132979,77.580444,1182.436534,78.145256,966.831651,0.074281,0.061991,0.663684,0.742242
5528,5529,Michigan St,152,0.0,51,0,big_ten,big_twelve,2.0,18,...,0.132979,0.201261,78.145256,966.831651,77.580444,1182.436534,0.061991,0.074281,0.742242,0.663684
5529,5530,Virginia,154,1.0,85,0,acc,big_twelve,1.0,32,...,0.155862,0.154279,63.842209,980.463322,77.580444,1182.436534,0.051226,0.073324,0.756856,0.711769


# GaTech's LRMC

Helper Function

In [60]:
def only_smaller(lst):
    lst = list(lst)
    smallest = lst[0]
    new_lst = []
    for item in lst:
        if item < smallest:
            smallest = item
        new_lst.append(smallest)
    return new_lst

## Getting the Rankings

In [80]:
temp = all_games_df[all_games_df['DayNum'] >= last_day_before_tournament+1][['Team', 'Opponent']]
temp = temp['Team'].to_list() + temp['Opponent'].to_list()
tour_teams = list(set(temp))

tour_games = all_games_df[all_games_df['DayNum'] >= last_day_before_tournament+1][['GameId', 'Team', 'Opponent', 'Win']]

tour_games['Winner'] = tour_games.apply(lambda x: x['Team'] if x['Win'] else x['Opponent'], axis=1)
tour_games_long = tour_games.copy()

In [62]:
ranking_dict = {}
unmatched_schools = []
for team in tour_teams:
    team_cleaned = team.replace(' ', '_')
    url = "https://www2.isye.gatech.edu/~jsokol/lrmc/TEAMS/{0}/".format(year) + team_cleaned + ".lrmc.sort0.php"

    try:
        aia_session = AIASession()
        context = aia_session.ssl_context_from_url(url)
        response = urlopen(url, context=context)
        soup = BeautifulSoup(response)
        metadata = soup.find_all('b')
        val = metadata[1]
        val = str(val)
        loc = val.find("=")
        val = val[loc+1:]
        loc = val.find("<")
        val = val[:loc]
        val = val.strip()
        val = int(val)
        ranking_dict[team] = val
        print(team, val)

    except:
        unmatched_schools.append(team)

St Louis 71
Northeastern 68
Iona 185
Iowa St 15
Wisconsin 13
Florida 30
Abilene Chr 160
Michigan 6
St Mary's CA 39
Kansas 18
Baylor 50
Colgate 100
N Dakota St 203
Houston 17
Virginia 4
Seton Hall 53
North Carolina 5
Oregon 32
UCF 54
Bradley 148
Syracuse 38
Yale 92
Belmont 47
Tennessee 7
Michigan St 3
Marquette 26
Florida St 16
Maryland 28
Texas Tech 8
Ohio St 45
Utah St 29
Montana 114
Gonzaga 1
Minnesota 51
Vermont 67
Purdue 11
UC Irvine 65
Virginia Tech 14
Kansas St 25
Duke 2
Old Dominion 90
Oklahoma 40
Washington 36
Iowa 43
Wofford 24
Arizona St 57
Mississippi 42
Buffalo 12
F Dickinson 196
Auburn 10
Kentucky 9
Nevada 19
LSU 23
Cincinnati 27
Gardner Webb 157
Mississippi St 21
Murray St 52
VA_Commonwealth 35
Villanova 22
Georgia St 97
N Kentucky 84
Liberty 64
Louisville 20
New Mexico St 41
St Louis 71
Northeastern 68
Iona 185
Iowa St 15
Wisconsin 13
Florida 30
Abilene Chr 160
Michigan 6
St Mary's CA 39
Kansas 18
Baylor 50
Colgate 100
N Dakota St 203
Houston 17
Virginia 4
Seton Hall 53


Check that all schools matched (the list should be empty)

In [78]:
unmatched_schools

['VCU']

In [81]:
tour_games_long['Team'] = tour_games_long['Team'].apply(lambda x: 'VA_Commonwealth' if x == 'VCU' else x)
tour_games_long['Opponent'] = tour_games_long['Opponent'].apply(lambda x: 'VA_Commonwealth' if x == 'VCU' else x)
tour_teams.remove('VCU')
tour_teams.append('VA_Commonwealth')

In [82]:
unmatched_schools = []
for team in ['VA_Commonwealth']:
    team_cleaned = team.replace(' ', '_')
    url = "https://www2.isye.gatech.edu/~jsokol/lrmc/TEAMS/{0}/".format(year) + team_cleaned + ".lrmc.sort0.php"

    try:
        aia_session = AIASession()
        context = aia_session.ssl_context_from_url(url)
        response = urlopen(url, context=context)
        soup = BeautifulSoup(response)
        metadata = soup.find_all('b')
        val = metadata[1]
        val = str(val)
        loc = val.find("=")
        val = val[loc+1:]
        loc = val.find("<")
        val = val[:loc]
        val = val.strip()
        val = int(val)
        ranking_dict[team] = val
        print(team, val)

    except:
        unmatched_schools.append(team)

VA_Commonwealth 35


In [83]:
unmatched_schools

[]

Check rankings

In [84]:
sorted(ranking_dict.items(), key=lambda x:x[1])

[('Gonzaga', 1),
 ('Duke', 2),
 ('Michigan St', 3),
 ('Virginia', 4),
 ('North Carolina', 5),
 ('Michigan', 6),
 ('Tennessee', 7),
 ('Texas Tech', 8),
 ('Kentucky', 9),
 ('Auburn', 10),
 ('Purdue', 11),
 ('Buffalo', 12),
 ('Wisconsin', 13),
 ('Virginia Tech', 14),
 ('Iowa St', 15),
 ('Florida St', 16),
 ('Houston', 17),
 ('Kansas', 18),
 ('Nevada', 19),
 ('Louisville', 20),
 ('Mississippi St', 21),
 ('Villanova', 22),
 ('LSU', 23),
 ('Wofford', 24),
 ('Kansas St', 25),
 ('Marquette', 26),
 ('Cincinnati', 27),
 ('Maryland', 28),
 ('Utah St', 29),
 ('Florida', 30),
 ('Oregon', 32),
 ('VA_Commonwealth', 35),
 ('Washington', 36),
 ('Syracuse', 38),
 ("St Mary's CA", 39),
 ('Oklahoma', 40),
 ('New Mexico St', 41),
 ('Mississippi', 42),
 ('Iowa', 43),
 ('Ohio St', 45),
 ('Belmont', 47),
 ('Baylor', 50),
 ('Minnesota', 51),
 ('Murray St', 52),
 ('Seton Hall', 53),
 ('UCF', 54),
 ('Arizona St', 57),
 ('Liberty', 64),
 ('UC Irvine', 65),
 ('Vermont', 67),
 ('Northeastern', 68),
 ('St Louis', 71

## LRMC Tournament Performance

In [85]:
tour_games = all_games_df[all_games_df['DayNum'] >= last_day_before_tournament+1][['GameId', 'Team', 'Opponent', 'Win']]

for team in tour_teams:
    tour_games.loc[tour_games['Team'] == team, 'Team_Rank'] = ranking_dict[team]
    tour_games.loc[tour_games['Opponent'] == team, 'Opp_Rank'] = ranking_dict[team]

tour_games['Winner'] = tour_games.apply(lambda x: x['Team'] if x['Win'] else x['Opponent'], axis=1)
tour_games_long = tour_games.copy()

for i in range(7):
    tour_games['Cum_Rank'] = tour_games.apply(lambda x: min(x['Team_Rank'], x['Opp_Rank']), axis=1)
    tour_games['Cum_Team_Rank_Final'] = 0
    tour_games['Cum_Opp_Rank_Final'] = 0
    for team in tour_teams:
        lst = tour_games[tour_games['Opponent'] == team][['Cum_Rank']].apply(only_smaller, axis=0)['Cum_Rank'].tolist()
        lst.insert(0, 1000)
        lst.pop()
        tour_games.loc[tour_games['Opponent'] == team, 'Cum_Opp_Rank_Final'] = lst

        lst2 = tour_games[tour_games['Team'] == team][['Cum_Rank']].apply(only_smaller, axis=0)['Cum_Rank'].tolist()
        lst2.insert(0, 1000)
        lst2.pop()
        tour_games.loc[tour_games['Team'] == team, 'Cum_Team_Rank_Final'] = lst2

    tour_games['Opp_Rank'] = tour_games.apply(lambda x : min(x['Cum_Opp_Rank_Final'], x['Opp_Rank']), axis=1)
    tour_games['Team_Rank'] = tour_games.apply(lambda x : min(x['Cum_Team_Rank_Final'], x['Team_Rank']), axis=1)

tour_games['Pred_Winner'] = tour_games.apply(lambda x: x['Team'] if x['Team_Rank'] < x['Opp_Rank'] else x['Opponent'], axis=1)
tour_games['Pred_Loser'] = tour_games.apply(lambda x: x['Team'] if x['Team_Rank'] > x['Opp_Rank'] else x['Opponent'], axis=1)

tour_games['Correct'] = tour_games.apply(lambda x: 1 if x['Pred_Winner'] == x['Winner'] else 0, axis=1)

tour_games_long['Pred_Winner'] = tour_games_long.apply(lambda x: x['Team'] if x['Team_Rank'] < x['Opp_Rank'] else x['Opponent'], axis=1)
tour_games_long['Pred_Loser'] = tour_games_long.apply(lambda x: x['Team'] if x['Team_Rank'] > x['Opp_Rank'] else x['Opponent'], axis=1)
tour_games_long['Correct'] = tour_games_long.apply(lambda x: 1 if x['Pred_Winner'] == x['Winner'] else 0, axis=1)
tour_games_long = tour_games_long[['GameId', 'Pred_Winner', 'Pred_Loser', 'Correct', 'Team_Rank', 'Opp_Rank']]
tour_games_long = tour_games_long.reset_index(drop=True)
tour_games_long = tour_games_long[tour_games_long.index % 2 == 0]
tour_games_long = tour_games_long.to_numpy()

tour_games = tour_games[['GameId', 'Pred_Winner', 'Pred_Loser', 'Correct', 'Team_Rank', 'Opp_Rank']]
tour_games = tour_games.reset_index(drop=True)
tour_games = tour_games[tour_games.index % 2 == 0]

tour_games = tour_games.to_numpy()

correct_count = 0
tour_teams_copy = tour_teams.copy()
for game in tour_games:
    if game[1] in tour_teams_copy and game[3]:
        correct_count += 1
        print(game[1], 'defeats', game[2])
    try:
        tour_teams_copy.remove(game[2])
    except:
        pass
correct_count

Kentucky defeats Abilene Chr
Auburn defeats New Mexico St
Maryland defeats Belmont
Michigan St defeats Bradley
Gonzaga defeats F Dickinson
Florida St defeats Vermont
Kansas defeats Northeastern
LSU defeats Yale
Michigan defeats Montana
Purdue defeats Old Dominion
Wofford defeats Seton Hall
Villanova defeats St Mary's CA
Buffalo defeats Arizona St
Tennessee defeats Colgate
Duke defeats N Dakota St
Virginia defeats Gardner Webb
Houston defeats Georgia St
North Carolina defeats Iona
Oklahoma defeats Mississippi
Texas Tech defeats N Kentucky
Virginia Tech defeats St Louis
Auburn defeats Kansas
Gonzaga defeats Baylor
Michigan defeats Florida
Florida St defeats Murray St
Kentucky defeats Wofford
LSU defeats Maryland
Michigan St defeats Minnesota
Purdue defeats Villanova
Texas Tech defeats Buffalo
Tennessee defeats Iowa
Virginia Tech defeats Liberty
North Carolina defeats Washington
Virginia defeats Oklahoma
Gonzaga defeats Florida St
Virginia defeats Oregon
Kentucky defeats Houston
Michigan 

40

The win total when picking all match-up winners at the start (like with a normal bracket) is shown above.
Below are the results when picking each match-up winner based on which teams made it to this stage in the tournament
(note that it does not take into consideration the teams' performance in earlier rounds of the tournament – all
features used in the models are taken prior to the start of the tournament). The top method is more difficult,
because even though you may have correctly predicted that Team B defeats Team C, if you had Team B losing to
Team A in the previous round, then you wouldn't have the option of choosing Team B in this round
(you would have likely chosen Team A – your prejected winner from the previous round, whom you thought would
be playing in this next game). If a top team loses early, but you had them winning in many future rounds,
then you no longer would have a chance at picking any of those games correctly.

In [86]:
correct_count = 0
tour_teams_copy = tour_teams.copy()
for game in tour_games_long:
    if game[3]:
        correct_count += 1
        print(game[1], 'defeats', game[2])
    try:
        tour_teams_copy.remove(game[2])
    except:
        pass
correct_count

Kentucky defeats Abilene Chr
Auburn defeats New Mexico St
Maryland defeats Belmont
Michigan St defeats Bradley
Gonzaga defeats F Dickinson
Florida St defeats Vermont
Kansas defeats Northeastern
LSU defeats Yale
Michigan defeats Montana
Purdue defeats Old Dominion
Wofford defeats Seton Hall
Villanova defeats St Mary's CA
Buffalo defeats Arizona St
Tennessee defeats Colgate
Duke defeats N Dakota St
Virginia defeats Gardner Webb
Houston defeats Georgia St
North Carolina defeats Iona
Oklahoma defeats Mississippi
Texas Tech defeats N Kentucky
Virginia Tech defeats St Louis
UCF defeats UCF
Auburn defeats Kansas
Gonzaga defeats Baylor
Michigan defeats Florida
Florida St defeats Murray St
Kentucky defeats Wofford
LSU defeats Maryland
Michigan St defeats Minnesota
Purdue defeats Villanova
Texas Tech defeats Buffalo
Duke defeats UCF
Houston defeats Ohio St
Tennessee defeats Iowa
Virginia Tech defeats Liberty
North Carolina defeats Washington
Virginia defeats Oklahoma
Oregon defeats UC Irvine
Gon

46

# Chalk Performance

When picking winners based on their seeds (chalk) we run into the issue of teams with the same seeds
playing each other in the Final Four and championship rounds. I've chosen to award half a point when
predicting the winner.

In [87]:
tour_games = all_games_df[all_games_df['DayNum'] >= last_day_before_tournament+1][['GameId', 'Team', 'Opponent', 'Win', 'Seed', 'Seed_Opp']]

tour_games['Winner'] = tour_games.apply(lambda x: x['Team'] if x['Win'] else x['Opponent'], axis=1)
tour_games['Pred_Winner'] = tour_games.apply(lambda x: "Tie" if x['Seed'] == x['Seed_Opp'] else x['Team'] if x['Seed'] < x['Seed_Opp'] else x['Opponent'], axis=1)
tour_games['Pred_Loser'] = tour_games.apply(lambda x: "Tie" if x['Seed'] == x['Seed_Opp'] else x['Team'] if x['Seed'] > x['Seed_Opp'] else x['Opponent'], axis=1)

tour_games['Correct'] = tour_games.apply(lambda x: 0.5 if x['Pred_Winner'] == 'Tie' else 1 if x['Pred_Winner'] == x['Winner'] else 0, axis=1)
tour_games = tour_games[['GameId', 'Pred_Winner', 'Pred_Loser', 'Correct', 'Seed', 'Seed_Opp']]
tour_games = tour_games.reset_index(drop=True)
tour_games = tour_games[tour_games.index % 2 == 0]
tour_games = tour_games.to_numpy()

In [88]:
tour_games

array([[5468, 'Kentucky', 'Abilene Chr', 1, 2.0, 15.0],
       [5469, 'Auburn', 'New Mexico St', 1, 12.0, 5.0],
       [5470, 'Syracuse', 'Baylor', 0, 8.0, 9.0],
       [5471, 'Maryland', 'Belmont', 1, 6.0, 11.0],
       [5472, 'Michigan St', 'Bradley', 1, 2.0, 15.0],
       [5473, 'Gonzaga', 'F Dickinson', 1, 1.0, 16.0],
       [5474, 'Nevada', 'Florida', 0, 7.0, 10.0],
       [5475, 'Florida St', 'Vermont', 1, 13.0, 4.0],
       [5476, 'Kansas', 'Northeastern', 1, 13.0, 4.0],
       [5477, 'Louisville', 'Minnesota', 0, 10.0, 7.0],
       [5478, 'LSU', 'Yale', 1, 14.0, 3.0],
       [5479, 'Marquette', 'Murray St', 0, 12.0, 5.0],
       [5480, 'Michigan', 'Montana', 1, 15.0, 2.0],
       [5481, 'Purdue', 'Old Dominion', 1, 3.0, 14.0],
       [5482, 'Wofford', 'Seton Hall', 1, 7.0, 10.0],
       [5483, 'Villanova', "St Mary's CA", 1, 6.0, 11.0],
       [5484, 'Buffalo', 'Arizona St', 1, 6.0, 11.0],
       [5485, 'Cincinnati', 'Iowa', 0, 10.0, 7.0],
       [5486, 'Tennessee', 'Colgate', 

Below are the results when picking each match-up winner based on which teams made it to this stage in the tournament
(note that it does not take into consideration the teams' performance in earlier rounds of the tournament – all
features used in the models are taken prior to the start of the tournament).

In [89]:
correct_count = 0
tour_teams_copy = tour_teams.copy()
for game in tour_games:
    correct_count += game[3]
    if game[3] == 1:
        print(game[1], 'defeats', game[2])
    elif game[3] == 0.5:
        print(game[1], 'ties', game[2])
    try:
        tour_teams_copy.remove(game[2])
    except:
        pass
correct_count

Kentucky defeats Abilene Chr
Auburn defeats New Mexico St
Maryland defeats Belmont
Michigan St defeats Bradley
Gonzaga defeats F Dickinson
Florida St defeats Vermont
Kansas defeats Northeastern
LSU defeats Yale
Michigan defeats Montana
Purdue defeats Old Dominion
Wofford defeats Seton Hall
Villanova defeats St Mary's CA
Buffalo defeats Arizona St
Tennessee defeats Colgate
Duke defeats N Dakota St
Virginia defeats Gardner Webb
Houston defeats Georgia St
North Carolina defeats Iona
Texas Tech defeats N Kentucky
Virginia Tech defeats St Louis
Gonzaga defeats Baylor
Michigan defeats Florida
Florida St defeats Murray St
Kentucky defeats Wofford
LSU defeats Maryland
Michigan St defeats Minnesota
Purdue defeats Villanova
Texas Tech defeats Buffalo
Duke defeats UCF
Houston defeats Ohio St
Tennessee defeats Iowa
Virginia Tech defeats Liberty
North Carolina defeats Washington
Virginia defeats Oklahoma
Oregon defeats UC Irvine
Gonzaga defeats Florida St
Virginia defeats Oregon
Duke defeats Virgin

43